## Workday Data Issue Semi-automation Tool (CCQ change)

This is a semi-automation tool which to facilate to performa the CCQ change requests
  
**Attention:**   This tool handles CCQ change request only



+ Step 1: **Check** if there's any open entries in the box folder. 
https://ibm.ent.box.com/file/731569093722

+ Step 2: **Validation** for cnums and field names
+ Step 3: _if there's open ticket in box file, we will **create the jira ticket** to handle._
https://jsw.ibm.com/browse/ODMODMR-91

+ Step 4: _**Update the box file** spreadsheet with correct jira ticket number, and check the status from "Not Started" to "in processing"_

+ Step 5: Read the spreadsheet and **generate the contents** for jira task (for all tickets with which status = 'in processing')

+ Step 6: Prepare **REK input** csv file

+ Step 7: **upload the CSV** file to MF server 

+ _Step 8: Execute the **RDSU load process** to load the REK table_
+ _Step 9: Execute the **CCQ change process** to perform the deletion_
+ _Step 10: **Confirm** the result_
+ _Step 11: Update the JIRA task and box spreadsheet to **close** the ticket_

In [ ]:
# Parameters
user = 'C943511'  # the userid in the JCL ODMSUBRJ, can be changed to Hans's ID e.g.
user_email = 'IBMCN.JIANGLEI -'
#user = 'NL62958'  # the userid in the JCL ODMSUBRJ, can be changed to Hans's ID e.g.
file_id = '731569093722'  # the box file id which holds the request spreadsheet
tbname = 'ODMT_EMPLOYEE'  # you can use the odm developer view if you don't have the access to E01
#tbname = 'ODMH_EMPLOYEE'  # you can use the odm developer view if you don't have the access to E01
#file_id = '766196407533'  # for TEST TEST TEST TEST

In [ ]:
import sys
sys.path.append('/odm_modules')
from common_func import odm_conn
sys.path.append('/app')
#from BOX import box_oauth as box
from BOX import box_jwt as box
import pandas as pd
import numpy as np
import datetime 
def cell_format(v):                                                             
    return str(v) if isinstance(v, (int)) else str(int(v)) if isinstance(v,(float)) else str(v.date()) if isinstance(v, (datetime.datetime)) else v.decode('utf-8') if isinstance(v, (bytes)) else '' if v is None else str(v).strip()                 

### Step 1 Read the file from box folder to check if there's any open items. 

+ If there's open entries, then show the open entries and do some pre-validation before processing
+ if there's no open entries, then STOP here. 

In [ ]:
client = box.get_box_client()
xlsx_file = client.file(file_id).get().name
print('File name for the data change request is:\n\t{}'.format(xlsx_file))
xlsx_content = client.file(file_id).content()
df = pd.read_excel(xlsx_content, header =2 ).fillna('').applymap(cell_format)
df = df.loc[(df.RSERNUM != '') & (df.C != '*'),:]
request_df = df.loc[df.status.isin(['Not Started', '']) ]
print('there are {} pending requests from Workday team'.format(request_df.shape[0]))
request_df

### Step 2 Validataion (only validate those ticket which are in status 'not started')

we perform the validation before raise the jira task  
+ step 2.1 CCQ and RSERNUM existence check


In [ ]:
request_df['condition'] = request_df.apply(lambda row: ' and '.join(["{} = '{}'".format(col, row[col]) for col in ['CCOUNTRY', 'CCOUNTRQ', 'RSERNUM']]), axis =1)
request_df['condition'] = "(" + request_df['condition'] + ")"
condition = '\nOR '.join(list(request_df.condition))
sql = 'select RCNUM, CCOUNTRY, CCOUNTRQ, RSERNUM, CACTIVE, DUPDATE, CFDRSRC \nFROM ODMPRD.{} \nWHERE {}'.format(tbname, condition)
print(sql)
with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
    result = odmprd_adhoc(sql)
    result_df = pd.DataFrame(result)
result_df

In [ ]:
request_df_x = request_df.merge(result_df, how = 'left', on = ['CCOUNTRY', 'CCOUNTRQ', 'RSERNUM'])
temp_df = request_df_x.loc[request_df_x.RCNUM.isnull()] 

if temp_df.shape[0] != 0: 
    print('{} entry could not be found in ODM database, please request Workday COE team to correct!'.format(temp_df.shape[0]))
    print('-'* 80)
    print(str(temp_df.loc[:, ['CCOUNTRY', 'CCOUNTRQ', 'RSERNUM']]))
else: 
    print('VALIDATION PASSED!')

### Step 3:  Create the JIRA task to address data change request (Manual  step)

if there's open ticket in box file, we will create the jira ticket to handle.   
Make sure the jira task is under the following epic   
https://jsw.ibm.com/browse/ODMODMR-91




### Step 4: Update the box file spreadsheet (Manual Step)  
+ Put the jira task number in the spreadsheet  
https://ibm.ent.box.com/file/731569093722
+ Put your name in the spreadsheet
+ Change the status from "Not Started" to "In processing"

### Step 5: Read the spreadsheet and generate the contents for jira task 

In [ ]:
# step 5.1 
# read all the ticket for ccq requests
client = box.get_box_client()
xlsx_file = client.file(file_id).get().name
xlsx_content = client.file(file_id).content()
df = pd.read_excel(xlsx_content,  header =2 ).fillna('').applymap(cell_format)
df = df.loc[(df.RSERNUM != '') & (df.C != '*'),:]
request_df = df.loc[(df.status == 'In Processing')]
print('there are {} pending CCQ change requests from Workday team'.format(request_df.shape[0]))
# get the Category information and task number
jira_nbrs = request_df.rtc_task_number.unique()
request_df

In [ ]:
print('there are {} jira ticket(s) for CCQ request'.format(len(jira_nbrs)))
for jira_nbr in jira_nbrs: 
    print('https://jsw.ibm.com/browse/{}'.format(jira_nbr))
if len(jira_nbrs) >= 2: 
    while True: 
        print('please tell which ticket to be handled this time? ({})'.format(', '.join(jira_nbrs)))
        jira_nbr = input()
        if jira_nbr in jira_nbrs:
            print('{} will be handled this time'.format(jira_nbr))
            break
        else: 
            print('{} is not a jira ticket number in status "in-processing", please try again'.format(jira_nbr))
if len(jira_nbrs) == 1: 
    jira_nbr = jira_nbrs[0]

rtc_num = 'RA' + jira_nbr.split('-')[1].zfill(5) 
request_df_current = request_df.loc[request_df.rtc_task_number == jira_nbr]
request_df_current
    

#### STEP 5.2 Copy the contents into jira task
also download the box spreadsheet and upload the spreadsheet to jira task 


In [ ]:
content_template = '''
*Zendesk Ticket*  
{zen_list}

&nbsp;


&nbsp;

|CCOUNTRY|CCOUNTRQ|RSERNUM|=>|NCOUNTRY|NCOUNTRQ|NSERNUM 
{request_entries}
&nbsp;   




Total impacted cnums are : {impacted_cnt}
&nbsp; 




*Query result before processing:* 
{sql}

&nbsp;



{result_col}
{result_val}
&nbsp;



We are going to use existing REK process to perform the deletion, 
no dry run is needed, the old data will be backuped in 2 datasets. 
&nbsp;
'''

In [ ]:
zen_list = '\n'.join(list(request_df_current.COE_zendesk_number.unique())) 
request_entries = '\n'.join(list(request_df_current.apply(lambda row: 
                                           ''.join(["|{}".format(item) for item in [
                                               row.CCOUNTRY, 
                                               row.CCOUNTRQ,
                                               row.RSERNUM,
                                               '=>', 
                                               row.NCOUNTRY, 
                                               row.NCOUNTRQ,
                                               row.NSERNUM,
                                           ]]), axis = 1)))
impacted_cnt = request_df_current.shape[0]
request_df_current['condition'] = request_df_current.apply(lambda row: ' and '.join(["{} = '{}'".format(col, row[col]) for col in ['CCOUNTRY', 'CCOUNTRQ', 'RSERNUM']]), axis =1)
request_df_current['condition'] = "(" + request_df_current['condition'] + ")"
condition = '\nOR '.join(list(request_df_current.condition))
sql = 'select RCNUM, CCOUNTRY, CCOUNTRQ, RSERNUM, CACTIVE, DUPDATE, CFDRSRC \nFROM ODMPRD.{} \nWHERE\n {}'.format(tbname, condition)
with odm_conn.odm_adhoc('prod') as odmprd_adhoc:
    result = odmprd_adhoc(sql)
    result_df = pd.DataFrame(result)
result_col = ''.join(["|{}".format(col) for col in result_df.columns])
result_val = []
for idx, row in result_df.iterrows():
    result_val.append( ''.join(["|{}".format(v)  for v in row.values]))
result_val = '\n'.join(result_val)
content = content_template.format(zen_list = zen_list, 
                                  request_entries = request_entries, 
                                  impacted_cnt = impacted_cnt, 
                                 sql = sql, 
                                 result_col = result_col,
                                 result_val = result_val)
print(content)

### STEP 6 Prepare REK input csv file

In [ ]:
# step 6.1 unload the current REK table
cols_list = 'CMODEL	CCOUNTRY	CCOUNTRQ	RSERNUM	NCOUNTRY	NCOUNTRQ	NSERNUM	RRTCTASK	FACTION	CLANGUAG	FDISCONT	QSORTSEQ	TCOUNTRY'
cols_list = cols_list.split()
with odm_conn.odm_adhoc('prod') as odmprd_adhoc: 
    result = odmprd_adhoc('select  {} from odmprd.odmt_emp_keychange order by QSORTSEQ'.format(','.join(cols_list)))
    df_rek = pd.DataFrame(result)
df_rek['C'] = 'R'
df_rek = df_rek.reindex(columns = ['C', *cols_list])
next_seq = int(df_rek.QSORTSEQ.max()) + 1

In [ ]:
# step 6.2 add the new entries for deletion request
ser = {}
for idx, row in request_df_current.iterrows():
    ser['C'] = 'R'
    ser['CMODEL'] = 'IBM'
    ser['CCOUNTRY'] = row.CCOUNTRY
    ser['CCOUNTRQ'] = row.CCOUNTRQ
    ser['RSERNUM'] = row.RSERNUM
    ser['NCOUNTRY'] = row.NCOUNTRY
    ser['NCOUNTRQ'] = row.NCOUNTRQ
    ser['NSERNUM'] = row.NSERNUM
    ser['RRTCTASK'] = jira_nbr.split('-')[-1]
    ser['FACTION'] = ''
    ser['QSORTSEQ'] = str(next_seq)
    ser['FDISCONT'] = ''  #####
    ser['TCOUNTRY'] = 'zen desk ticket: {}'.format(row.COE_zendesk_number)
    df_rek = df_rek.append(ser, ignore_index = True).fillna('')
df_rek = df_rek.applymap(cell_format)

In [ ]:
# step 6.3 write csv file and add the first line
now = str(datetime.datetime.today().date())
df_rek.to_csv('temp.csv', index = False)
rek_filename = 'REK_{}.csv'.format(''.join(now.split('-')))
with open(rek_filename, 'w') as fw: 
    fw.write('T,REK - Employee Primary key change (ODMT_EMP_KEYCHANGE),,,,,,,,,,,,\n')
    fw.write(open('temp.csv', 'r').read())
print('{} \nfile is created in local machine which is ready to be uploaded to MF server'.format(rek_filename))

+++++++
### STEP 7 upload the CSV file to MF server

In [ ]:
# step 7 upload the csv file to stfmvs1 server
from common_func import odm_ftp
with odm_ftp.odm_ftp_conn('put') as odm_put_file:
    server_file = 'ODMAP.RES.ZZ.RDMCSV.IN.EM'
    odm_put_file(fm=rek_filename, to= server_file)
    print( 'file {} is uploade in server side...'.format(server_file))

+++++++
### STEP 8 _Execute the RDSU load process to load the REK table_

### STEP 9 _Execute the CCQ change process to perform the deletion_

### STEP 10 _Confirm the result_


### STEP 11 _Update the JIRA task and box spreadsheet to close the ticket_
